In [15]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
import pickle
import dask.dataframe as dd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import subprocess
import random
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import numpy as np
import cvxpy as cp

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from itertools import combinations

In [ ]:
safedata/ath_evo/grenephase1-data/merged_tables/merged_hapFIRE_delta_p_normed.txt
merged_hapFIRE_delta_p_LDpruned

In [31]:
path_ldp_af = '/carnegie/nobackup/scratch/xwu/grenet/merged_frequency/merged_hapFIRE_delta_p_LDpruned.txt'

In [32]:
allele_freq = dd.read_csv(path_ldp_af, sep = '\t')

In [33]:
allele_freq = allele_freq.compute() 
allele_freq = allele_freq.reset_index(drop=True)

In [34]:
final_gen = pd.read_csv('../final_gen.csv')['sample_name'].to_list()

In [35]:
allele_freq = allele_freq[final_gen]

In [36]:
sites = allele_freq.columns.str.split('_').str[0].unique()

In [37]:
combinations

itertools.combinations

In [38]:
r2_af = {}
for i in sites:
    # Extract columns that start with the site prefix
    site1 = [col for col in allele_freq.columns if col.startswith(f'{i}_')]
    
    # Initialize a dictionary to store R^2 values
    r2_values = []
    
    # Calculate R^2 for each pair of columns
    for (col1, col2) in combinations(site1, 2):
        # Extract the columns as X and y
        X = allele_freq[[col1]]
        y = allele_freq[col2]
        
        # Fit linear regression model
        model = LinearRegression().fit(X, y)
        
        # Calculate R^2 and store it
        r2 = model.score(X, y)
        r2_values.append(r2)
    
    # Calculate the mean of these R^2 values
    mean_r2 = np.mean(r2_values)
    
    # Output the results
    r2_af[i] = mean_r2

In [39]:
r2_af = pd.DataFrame(list(r2_af.items()), columns=['site', 'r2'])


In [40]:
len(r2_af.sort_values('r2'))

31

In [41]:
r2_af.sort_values('r2')

,site,r2
5,9,0.010687
1,2,0.011304
8,12,0.014705
23,48,0.014705
7,11,0.017004
24,49,0.019911
12,25,0.021523
11,24,0.021716
14,27,0.022002
30,60,0.023933


In [48]:
corr_af = {}
for i in sites:
    site1 = [col for col in allele_freq.columns if col.startswith(f'{i}_')]
    corr_site1 = allele_freq[site1].corr()
    
    upper_tri_indices = np.triu_indices_from(corr_site1, k=1)
    
    # Extract the values from the upper triangle
    upper_tri_values = corr_site1.values[upper_tri_indices]

    corr_af[i] = upper_tri_values.mean()

In [49]:
corr_af = pd.DataFrame(list(corr_af.items()), columns=['site', 'corr'])

In [51]:
corr_af.sort_values('corr')

,site,corr
1,2,0.036371
23,48,0.038320
5,9,0.042392
7,11,0.048359
14,27,0.049603
8,12,0.056271
30,60,0.070908
11,24,0.084145
6,10,0.090923
24,49,0.092499


In [55]:
higher_corr = corr_af[corr_af['corr'] > 0.1]

In [ ]:
## from now on im onyl goign t owork with sites with a correlation at least hgiher than 0.1 

In [90]:
higher_corr_sites = higher_corr['site'].astype(int).to_list()

In [81]:
final_gen_samples = pd.read_csv('../final_gen.csv')

In [73]:
from sklearn.model_selection import train_test_split

In [91]:
#from sklearn.model_selection import train_test_split
#unique_sites_final_gen

train_size = 16
test_size = 4

# List to hold the splits
splits = []

for _ in range(100):
    train, test = train_test_split(higher_corr_sites, train_size=train_size, test_size=test_size, random_state=None)
    splits.append((train, test))
    

#splits = pd.DataFrame(splits)

#splits.columns = ['train', 'test']

#splits.to_csv('splits_l1o.csv')



In [92]:
with open('splits_sites.pkl', 'wb') as f:
    pickle.dump(splits, f)

sample_splits = []
for train_indices, test_indices in splits:
    train_samples = final_gen_samples[final_gen_samples['site'].isin(train_indices)]['sample_name'].to_list()
    test_samples = final_gen_samples[final_gen_samples['site'].isin(test_indices)]['sample_name'].to_list()
    sample_splits.append((train_samples, test_samples))

with open('splits_samples.pkl', 'wb') as f:
    pickle.dump(sample_splits, f)

In [93]:
with open('splits_samples.pkl', 'rb') as file:
    splits_samples = pickle.load(file)

len(splits_samples) ## 100

100